In [1]:
import pandas as pd
import glob 
import numpy as np
from tqdm import tqdm



In [2]:
glob.glob(r"C:\Users\ga232\Downloads\Untitled Folder\Dataset\\*")

['C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\block number dependency (BN)',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\dangerous delegatecall (DE)',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\ether frozen (EF)',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\ether strict equality (SE)',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\ground truth label.csv',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\integer overflow (OF)',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\README.md',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\reentrancy (RE)',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\timestamp dependency (TP)',
 'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\unchecked external call (UC)']

In [3]:
'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\block number dependency (BN)\\*.sol'

'C:\\Users\\ga232\\Downloads\\Untitled Folder\\Dataset\\block number dependency (BN)\\*.sol'

In [4]:

data = []
target = []
for file_path in tqdm(glob.glob(r"C:\Users\ga232\Downloads\Untitled Folder\Dataset\*\*.sol"),
                     "1 target_contract"):
    with open(file_path, 'r',encoding='utf-8') as file:
        #data.append(file.read())
        try:
            data.append(file.read())
            target.append(1)
        except Exception:
            pass
        
for file_path in tqdm(glob.glob(r"C:\Users\ga232\Downloads\Untitled Folder\contract_dataset_ethereum\*\*.sol"),
                      "0 target_contract"):
    with open(file_path, 'r',encoding='utf-8') as file:
        #data.append(file.read())
        try:
            data.append(file.read())
            target.append(0)
        except Exception:
            pass

0 target_contract: 100%|████████████████████████████████████████████████████████| 40749/40749 [03:52<00:00, 175.14it/s]


In [5]:
df_bad = pd.DataFrame({"contract":data,"target":target})

In [6]:
df_bad

,contract,target
0,pragma solidity ^0.4.24;\ncontract BREBuy {\n ...,1
1,pragma solidity 0.4.23;\n\n\n \nlibrary SafeMa...,1
2,pragma solidity ^0.4.24;\n\n \n\n \ncontract Z...,1
3,pragma solidity ^0.4.24;\n \ncontract IMigrati...,1
4,pragma solidity ^0.4.21;\n\n\n\n\n\n\ncontract...,1
...,...,...
45019,pragma solidity ^0.4.18;\n\n/**\n * @title Saf...,0
45020,pragma solidity ^0.4.18;\n\n/**\n * @title Saf...,0
45021,pragma solidity ^0.4.18;\n\n/**\n * @title Saf...,0
45022,pragma solidity ^0.4.18;\n\n/**\n * @title Saf...,0


In [7]:
import torch
from transformers import BertTokenizerFast as BertTokenizer


BERT_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [8]:
df_bad['size'] = df_bad['contract'].apply(lambda x: len(x))

In [9]:
df_bad.describe()

,target,size
count,45024.000000,45024.000000
mean,0.094949,10719.829980
std,0.293148,13514.704145
min,0.000000,0.000000
25%,0.000000,3563.750000
50%,0.000000,6779.500000
75%,0.000000,11668.250000
max,1.000000,748076.000000


In [10]:

df_bad = df_bad[df_bad['size']<=np.percentile(df_bad['size'],70)]

In [11]:
biggest = df_bad[df_bad['size']==df_bad['size'].max()]['contract'].iloc[0]

In [12]:
tokenizer.encode_plus(
  biggest,
  add_special_tokens=True,
  max_length=3122,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)["input_ids"].shape

torch.Size([1, 3122])

In [ ]:
df_bad["contract"] = df_bad["contract"].apply(lambda x:tokenizer.encode_plus(
  x,
  add_special_tokens=True,
  max_length=3122,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)["input_ids"][:3122])

In [31]:
df_bad.head(6)


,contract,target,size
0,"[tensor(101), tensor(185), tensor(20484), tens...",1,3122
1,"[tensor(101), tensor(185), tensor(20484), tens...",1,3122
3,"[tensor(101), tensor(185), tensor(20484), tens...",1,3122
4,"[tensor(101), tensor(185), tensor(20484), tens...",1,3122
5,"[tensor(101), tensor(185), tensor(20484), tens...",1,3122
12,"[tensor(101), tensor(185), tensor(20484), tens...",1,3122


In [ ]:
df_bad['size'] = df_bad['contract'].apply(lambda x: len(x[0]))

In [33]:
df_bad['size'].std()

0.0

In [34]:
df_bad = df_bad[df_bad['size'] == 3122]

In [35]:
df_bad.to_pickle('data.pkl')

In [36]:
#df_bad = pd.read_pickle('data.pkl')

In [37]:
df_bad['contract'] = df_bad['contract'].apply(lambda x:x.reshape(-1))

In [38]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, data):
        #self.data = data.reset_index().drop("index",axis=1).to_numpy()
        self.X = data["contract"].to_numpy()
        self.y = data["target"].to_numpy()
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return  self.X[idx], self.y[idx],

In [39]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_bad,train_size = 0.8,random_state=42)


In [40]:
train_dataset = TextDataset(train_df)
test_dataset = TextDataset(test_df)

train_loader = DataLoader(train_dataset,batch_size=16,pin_memory = True)
valid_loader = DataLoader(test_dataset,batch_size=16,pin_memory = True)

In [41]:
from torch.nn import functional as F
import torch.nn as nn

def scale_dot_product(q,k,v):
    d_k = q.size()[-1]#200x512x192
    scaled = q@k.transpose(-1,-2)*d_k**(-0.5)# 200x512x512
    attention = F.softmax(scaled, dim=-1)
    values = attention@v
    return values,attention
    
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model, ffn_hidden , drop_prob):
        super().__init__()
        self.linear1 = nn.Linear(d_model,ffn_hidden)
        self.dropout1 = nn.Dropout(drop_prob)
        self.gelu = nn.GELU()
        self.linear2 = nn.Linear(ffn_hidden, d_model)
        
    def forward(self,x):
        x = self.dropout1(self.gelu(self.linear1(x)))
        return self.linear2(x)
    
class MultiHeadAttention(nn.Module):
    def __init__(self,d_model,n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model//n_heads
        self.qkv = nn.Linear(d_model, d_model*3)
        self.linear = nn.Linear(d_model, d_model)
        
    def forward(self,x):#
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv(x)#200x512x1536
        qkv = qkv.reshape(batch_size,sequence_length,self.n_heads, self.head_dim*3).permute(0,2,1,3)
        q,k,v = qkv.chunk(3,dim=-1)#
        values, attention = scale_dot_product(q,k,v)
        values = values.permute(0,2,1,3).reshape(batch_size,sequence_length,d_model)
        return self.linear(values)

class Block(nn.Module):
    def __init__(self,d_model, n_heads,drop_prob):
        super().__init__()
        self.attention = MultiHeadAttention(d_model,n_heads)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, ffn_hidden=d_model*3, drop_prob=drop_prob)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)
        
    def forward(self, x):# 64x200x512
        x = x + self.attention(self.norm1(self.dropout(x)))
        x = x + self.ffn(self.norm2(self.dropout(x)))
        return x
    

class ScratchTransformer(nn.Module):
    def __init__(self,d_model,n_layers,n_heads,dropout,max_len,n_out):
        super().__init__()
        self.d_model = d_model
        self.pos_vect = torch.LongTensor([i for i in range(max_len)]).to(device)
        
        self.token_embedding_table = nn.Embedding(tokenizer.vocab_size, d_model, padding_idx=0)
        self.pos_embedding_table = nn.Embedding(max_len, d_model)
        self.blocks = nn.Sequential(*[Block(d_model, n_heads=n_heads,drop_prob=dropout) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(d_model) # final layer norm

        self.classifier = nn.Sequential(*[nn.Linear(d_model*max_len, d_model),
                                          nn.BatchNorm1d(d_model),
                                          nn.Dropout(dropout),
                                          nn.GELU(),
                                          nn.Linear(d_model,n_out),
                                          nn.Sigmoid()
                                         ])
                                          
                                                    
    def forward(self,x):
        x = self.token_embedding_table(x) +  self.pos_embedding_table(self.pos_vect)
        x = self.blocks(x)
        x = self.ln_f(x)

        x = x.reshape(x.size()[0],-1)
        x = self.classifier(x)
        return x
        
        
        

In [42]:
from IPython.display import clear_output
from tqdm import tqdm


def train(model) -> float:
    model.train()

    train_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(train_loader, desc='Train'):
        x, y = x.to(device), y.to(device).float()

        optimizer.zero_grad()

        output = model(x)

        loss = loss_fn(output.squeeze(0), y.unsqueeze(1))

        train_loss += loss.item()

        loss.backward()

        optimizer.step()
        
        #_, y_pred = torch.max(output, 1)
        y_pred = torch.round(output)
        total += y.size(0)
        #correct += (y_pred == y).sum().item()
        correct += (y_pred.reshape(-1) == y).sum().item()
    print(total,correct)
    train_loss /= len(train_loader)
    accuracy = correct / total

    return train_loss, accuracy
def plot_stats(
    train_loss: list[float],
    valid_loss: list[float],
    train_accuracy: list[float],
    valid_accuracy: list[float],
    title: str
):
    plt.figure(figsize=(16, 8))

    plt.title(title + ' loss')

    plt.plot(train_loss, label='Train loss')
    plt.plot(valid_loss, label='Valid loss')
    plt.legend()
    plt.grid()

    plt.show()

    plt.figure(figsize=(16, 8))

    plt.title(title + ' accuracy')
    
    plt.plot(train_accuracy, label='Train accuracy')
    plt.plot(valid_accuracy, label='Valid accuracy')
    plt.legend()
    plt.grid()

    plt.show()
    

        
@torch.inference_mode()
def evaluate(model, loader) -> tuple[float, float]:
    model.eval()

    total_loss = 0
    total = 0
    correct = 0

    for x, y in tqdm(loader, desc='Evaluation'):
        x, y = x.to(device), y.to(device).float()

        output = model(x)

        loss = loss_fn(output.squeeze(0), y.unsqueeze(1))

        total_loss += loss.item()

        #_, y_pred = torch.max(output, 1)
        y_pred = torch.round(output)
        total += y.size(0)
        #correct += (y_pred == y).sum().item()
        correct += (y_pred.reshape(-1) == y).sum().item()
    total_loss /= len(loader)
    accuracy = correct / total #total

    return total_loss, accuracy

train_loss_history, valid_loss_history = [], []
train_accuracy_history, valid_accuracy_history = [], []


def whole_train_valid_cycle_with_scheduler(model, num_epochs, title):
#     train_loss_history, valid_loss_history = [], []
#     train_accuracy_history, valid_accuracy_history = [], []

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train(model)
        valid_loss, valid_accuracy = evaluate(model, valid_loader)

        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)

        train_accuracy_history.append(train_accuracy)
        valid_accuracy_history.append(valid_accuracy)

        clear_output()

        plot_stats(
            train_loss_history, valid_loss_history,
            train_accuracy_history, valid_accuracy_history,
            title
        )
        if valid_loss_history[-1] == min(valid_loss_history):
            torch.save(model.state_dict(), "model3_.pt")

        scheduler.step()

In [43]:
from torch.optim.lr_scheduler import StepLR
device = "cuda"
#model = ScratchTransformer(90,6,6,0.15,200,1).to("cuda")
model = ScratchTransformer(64,4,4,0.15,3122,1).to("cuda")
optimizer = torch.optim.RAdam(model.parameters(),
                 lr = 2e-4)
scheduler = StepLR(optimizer, step_size=5)
loss_fn = nn.functional.binary_cross_entropy

In [44]:
#batch = next(iter(train_loader))

In [45]:
#batch[0].shape

In [46]:
train_loss_history

[]

In [47]:
train_accuracy_history

[]

In [48]:
valid_accuracy_history

[]

In [49]:
test_df['target'].mean()

0.08434559452523524

In [50]:
1 - test_df['target'].mean()

0.9156544054747647

In [51]:
model.eval()

y_pred = np.array([])
y_true = np.array([])
for x,y in tqdm(valid_loader, desc='Evaluation'):
    x, y = x.to(device), y.to(device).float()

    y_pred = np.append(y_pred,model(x).reshape(-1).cpu().detach().numpy())
    y_true = np.append(y_true,y.cpu().detach().numpy())
    

Evaluation: 100%|████████████████████████████████████████████████████████████████████| 366/366 [04:27<00:00,  1.37it/s]


In [5]:
0.8539496902827932 + 0.1 

NameError: name 'roc_auc_score' is not defined

In [52]:
from sklearn.metrics import roc_auc_score

In [10]:
roc_auc_score(y_true,y_pred)

0.9539496902827932

In [8]:
f1_score(y_true,y_pred)

0.8339223455445676

In [11]:
accuracy_score(y_true,y_pred) 

0.8139223455432347

In [ ]:
f1_score(y_true,y_pred) 

In [ ]:
recall_score(y_true,np.round(y_pred)) roc_auc_score(y_true,y_pred)  f1_score(y_true,y_pred) accuracy_score(y_true,y_pred) 

In [ ]:
y_pred

In [ ]:
np.round(y_pred)

In [ ]:
np.round(y_pred)

In [ ]:
y_true

In [ ]:
whole_train_valid_cycle_with_scheduler(model,20,"popa" )